### Manual Curation of Model Aliases

In [21]:
import biosapi
import json
from biosapi import BIOS
from biosapi.bios_model_alias_mapper import BiosModelAliasMapper
from biosapi.bios_model_reaction import BiosModelReaction
import cobrakbase
#%run ../biosapi/api.py

In [2]:
modelseed_local = cobrakbase.modelseed.from_local('../../../ModelSEEDDatabase')

load: ../../../ModelSEEDDatabase/Biochemistry/reactions.tsv
load: ../../../ModelSEEDDatabase/Biochemistry/compounds.tsv
load: ../../../ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [3]:
bios = BIOS()

In [76]:
MODEL_CPD_MAPPING = None
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/cpd_mapping_cache4.json', 'r') as f:
    MODEL_CPD_MAPPING = json.loads(f.read())
MODEL_RXN_MAPPING = None
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/rxn_mapping_cache4.json', 'r') as f:
    MODEL_RXN_MAPPING = json.loads(f.read())

In [65]:
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__MIT']['M_PYRUVATE__91__CCO__45__MIT__93__']

['cpd00020']

In [73]:
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__CYTOSOL']['M_WATER'] = ['cpd00001']
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__CYTOSOL']['M_OH'] = ['cpd15275']
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__CYTOSOL']['M_AMMONIUM'] = ['cpd00013']
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__CYTOSOL']['M_AMMONIA'] = []
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__MIT']['M_AMMONIUM__91__CCO__45__MIT__93__'] = ['cpd00013']
MODEL_CPD_MAPPING['iJDZ836']['CCO__45__MIT']['M_AMMONIA__91__CCO__45__MIT__93__'] = []

In [74]:
MODEL_CPD_MAPPING['iAL1006']['C_c']['M_G6P'] = ['cpd19006']

In [75]:
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/cpd_mapping_cache4.json', 'w') as f:
    f.write(json.dumps(MODEL_CPD_MAPPING))

In [59]:
MODEL_RXN_MAPPING['iJDZ836']['TRANS__45__RXN2T__45__215__45__L2R__91__CCO__45__MIT__93__']

['rxn05469']

In [58]:
model_id = 'iJDZ836'
for cmp in MODEL_CPD_MAPPING[model_id]:
    for spi_id in MODEL_CPD_MAPPING[model_id][cmp]:
        if spi_id in cpd_mapping:
            for db_id in cpd_mapping[spi_id]:
                if not db_id in MODEL_CPD_MAPPING[model_id][cmp][spi_id]:
                    MODEL_CPD_MAPPING[model_id][cmp][spi_id].append(db_id)


In [20]:
import pandas as pd
target_model_id = 'iJDZ836'
df_manual_cpd = pd.read_csv('/Users/fliu/workspace/jupyter/data/fungi/cpd_mapping_curated.tsv', sep='\t');
for row_id, d in df_manual_cpd.iterrows():
    model_id = d['model_id']
    if model_id == target_model_id:
        for cpd_id in d.keys():
            if not cpd_id == 'model_id':
                if not pd.isna(d[cpd_id]):
                    seed_cpd = modelseed_local.get_seed_compound(cpd_id)
                    uid = mapper.sid_to_spi_id[d[cpd_id]]
                    resp = bios.set_annotation_model_species(target_model_id, uid, 'ModelSeed', cpd_id, 'fliu', 4)
                    #print(model_id, uid, d[cpd_id], cpd_id, sid_to_spi[d[cpd_id]]['name'], seed_cpd.name)

In [14]:
model_id = 'iJDZ836'
mapper = BiosModelAliasMapper(bios)
mapper.load_model(model_id)
sid_to_spi = {}
for o in mapper.model_species:
    sid_to_spi[o['id']] = o

In [15]:
cpd_mapping = mapper.get_cpd_mapping('ModelSeed', 1)

In [27]:
for k in cpd_mapping:
    if len(cpd_mapping[k]) > 0:
        print(k, cpd_mapping[k])
    break

M_SORBITOL__91__CCO__45__EXTRACELLULAR__93__ {'cpd00588'}


In [12]:
bigg_to_seed = {}
for seed_id in modelseed_local.compound_aliases:
    if 'BiGG' in modelseed_local.compound_aliases[seed_id]:
        for i in modelseed_local.compound_aliases[seed_id]['BiGG']:
            bigg_to_seed[i] = seed_id
meta_to_seed = {}
for seed_id in modelseed_local.compound_aliases:
    if 'MetaCyc' in modelseed_local.compound_aliases[seed_id]:
        for i in modelseed_local.compound_aliases[seed_id]['MetaCyc']:
            meta_to_seed[i] = seed_id

In [53]:
meta_to_seed['SORBITOL']

'cpd00588'

In [16]:
ban = set(['OH', 'AMMONIA', 'AMMONIUM'])
for k in cpd_mapping:
    search_str = k.split('__91__CCO__45__EXTRACELLULAR__93__')[0]
    search_str = search_str.split('__91__CCO__45__VAC__45__LUM__93__')[0]
    search_str = search_str.split('__91__CCO__45__GLYOXYSOME__93__')[0]
    search_str = search_str.split('__91__CCO__45__MIT__93__')[0]
    search_str = search_str[2:]
    search_str = search_str.replace('__45__', '-')
    search_str = search_str.replace('__43__', '+')
    if not search_str in ban and search_str in meta_to_seed:
        if not meta_to_seed[search_str] in cpd_mapping[k]:
            print(search_str, cpd_mapping[k], meta_to_seed[search_str])

N-ACETYL-D-GLUCOSAMINE-1-P {'cpd02611'} cpd19031
CPD-10725 set() cpd03014
CPD-3569 set() cpd24418
CPD-13390 set() cpd23854
CPD-13404 set() cpd11593
CPD-13406 set() cpd11589
CPD-10814 set() cpd22832
ALA-GLY set() cpd11585
NA+ {'cpd00971'} cpd16174
DOCOSANOATE set() cpd05196
18-OXOOLEATE set() cpd20870
CPD-7838 set() cpd20876
CPD-729 set() cpd00903
CPD-728 set() cpd02846
LINOLENIC_ACID set() cpd03850
13-HYDROPEROXYOCTADECA-911-DIENOATE set() cpd02873
CPD-725 set() cpd02907
CAMP set() cpd00446
CPD-3710 set() cpd01991
CPD-3723 set() cpd01939
CAMP set() cpd00446
CPD-3710 set() cpd01991
CPD-3723 set() cpd01939
THREO-DS-ISO-CITRATE {'cpd00260'} cpd00353
L-CYSTATHIONINE {'cpd00424'} cpd19019
3-HYDROXY-L-KYNURENINE {'cpd02065'} cpd01804
CPD-7838 set() cpd20876
18-OXOOLEATE set() cpd20870
2-HYDROXY-3-KETO-5-METHYLTHIO-1-PHOSPHOP set() cpd11296
CELLOBIOSE {'cpd00158'} cpd03845
5-UREIDO-4-IMIDAZOLE_CARBOXYLATE set() cpd03280
4-AMINO-5-IMIDAZOLE_CARBOXYLATE set() cpd03281
13-HYDROPEROXYOCTADECA-911

In [377]:
spi_uid = str(sid_to_id['M_nh4_c'])
for uid in model_species_annotation[spi_uid]['annotation']:
    annotation = model_species_annotation[spi_uid]['annotation'][uid]
    if annotation['database'] == 'ModelSeed':

        print(annotation, score, max_score)

{'entry': 'cpd19013', 'database': 'ModelSeed', 'link': {'updated_at': 1545123696550, 'created_at': 1545123631265, 'authors': 'system:3'}} {'system': '3'} 3
{'entry': 'cpd00013', 'database': 'ModelSeed', 'link': {'updated_at': 1587921027276, 'created_at': 1530148385303, 'authors': 'fliu:5;system:3'}} {'fliu': '5', 'system': '3'} 5


In [19]:
for spi_uid in model_species_annotation:
    spi_uid

In [20]:
spi_uid

'3153423'

In [73]:
from cobrakbase.modelseed.stoich_integration import multi_hasher, CStoichiometryHashLibrary
from cobrakbase.modelseed.stoich_integration import StoichiometryHashLibrary, single_hasher
hfunc = lambda x : single_hasher(x, ['cpd00067_c0', 'cpd00067_e0'])
hlib = StoichiometryHashLibrary(hfunc)

In [71]:
hlib = StoichiometryHashLibrary(hfunc)
for o in model_reactions:
    model_reaction = BiosModelReaction(o)
    stoichiometry = model_reaction.get_stoichiometry()
    hlib.hash_stoichiometry(model_reaction.id, stoichiometry)

In [74]:
def is_standard_match(mresult):
    for o in mresult:
        if o[0] == 'std' and o[0] == 'std':
            return True
        #print(o)
    return False

def is_reverse_match(mresult):
    for o in mresult:
        if o[0] == 'std' and o[0] == 'rev':
            return True
        if o[0] == 'rev' and o[0] == 'std':
            return True
        #print(o)
    return False

def is_exclude_match(mresult):
    for o in mresult:
        if o[0] == 'no_h_std' or o[0] == 'no_h_std':
            return True
        if o[0] == 'no_h_rev' or o[0] == 'no_h_rev':
            return True
    return False

In [ ]:


exclude = set(['ATPM', 'BIOMASS_Ecoli'])
for rxn in model.reactions:
     if rxn.id.startswith('EX_'):
        exclude.add(rxn.id)
mapping_rxn = {}
for model_rxn in model.reactions:
    stoichiometry = dict(map(lambda p : (ooo[p[0].id] + '_' + p[0].compartment + '0', p[1]), model_rxn.metabolites.items()))
    m = hlib.match(stoichiometry)
    if not len(m) == 0:
        for match_id in m:
            if is_standard_match(m[match_id]):
                mapping_rxn[model_rxn.id] = (match_id, 'standard')
            elif is_reverse_match(m[match_id]):
                mapping_rxn[model_rxn.id] = (match_id, 'reverse')
            elif len(model_rxn.compartments) == 1 and is_exclude_match(m[match_id]):
                mapping_rxn[model_rxn.id] = (match_id, 'exclude')
            else:
                print('!!', model_rxn)
                #print(model_rxn.id, match_id)
    else:
        #print('xx', model_rxn)
        pass

In [75]:
compartment_config = cobrakbase.core.utils.get_compartment_config(r.data, modelseed.get_seed_reaction(seed_id))

NameError: name 'r' is not defined

In [86]:
model_reaction = BiosModelReaction(list(filter(lambda x : x['id'] == 'R_PYRt2m', model_reactions))[0])
s = model_reaction.get_stoichiometry()

In [146]:
cs = {}
for k in s:
    cs[(sid_to_spi[k]['id'], sid_to_spi[k]['compartment'])] = s[k]
cs
#replace with seed_ids

{('M_pyr_c', 'c'): -1.0,
 ('M_h_c', 'c'): -1.0,
 ('M_pyr_m', 'm'): 1.0,
 ('M_h_m', 'm'): 1.0}

In [88]:
sid_to_spi = {}
for o in model_species:
    sid_to_spi[o['id']] = o

In [92]:
rxn05469

'm'

In [28]:
import copy
from itertools import permutations
def get_permutations(n):
    perm = permutations(range(n), n) 
    return list(perm)

class StoichiometryHashLibrary:
    
    def __init__(self):
        self.hash_functions = {}
        self.all_hashes = {}
        self.rxn_to_hash = {}
        
    def hash_stoichiometry(self, rxn_id, stoich):
        self.rxn_to_hash[rxn_id] = {}
        for fn_hash_id in self.hash_functions:
            fn_hash = self.hash_functions[fn_hash_id]
            hash_val = fn_hash(stoich)
            self.rxn_to_hash[rxn_id][fn_hash_id] = hash_val
            if not fn_hash_id in self.all_hashes:
                self.all_hashes[fn_hash_id] = {}
            if not hash_val in self.all_hashes[fn_hash_id]:
                self.all_hashes[fn_hash_id][hash_val] = set()
            self.all_hashes[fn_hash_id][hash_val].add(rxn_id)
            
    def match(self, stoichiometry):
        match = {}
        s_hash = self.func_hash(stoichiometry)
        for hash_type in s_hash:
            hash_val = s_hash[hash_type]
            for hash_type_lib in self.all_hashes:
                if hash_val in self.all_hashes[hash_type_lib]:
                    for rxn_id_match in self.all_hashes[hash_type_lib][hash_val]:
                        if not rxn_id_match in match:
                            match[rxn_id_match] = set()
                        match[rxn_id_match].add((hash_type, hash_type_lib, hash_val))
        return match
    
class CStoichiometryHashLibrary(StoichiometryHashLibrary):
    
    def match(self, cstoichiometry):
        match = {}
        cmps = list(set(map(lambda x : x[1], cstoichiometry)))
        index_tests = get_permutations(len(cmps))
        
        for t in index_tests:
            replace = {}
            for i in range(len(cmps)):
                replace[cmps[i]] = str(t[i])
                
            tstoichiometry = copy.deepcopy(cstoichiometry)
            tstoichiometry = dict(map(lambda x : ((x[0][0], replace[x[0][1]]), x[1]), tstoichiometry.items()))
            
            
            for fn_hash_id in self.hash_functions:
                fn_hash = self.hash_functions[fn_hash_id]
                hash_val = fn_hash(tstoichiometry)
                #print(fn_hash_id, hash_val)
                for lib_fn_hash_id in self.all_hashes:
                    if hash_val in self.all_hashes[lib_fn_hash_id]:
                        for rxn_id_match in self.all_hashes[lib_fn_hash_id][hash_val]:
                            if not rxn_id_match in match:
                                match[rxn_id_match] = set()
                            replace_str = ';'.join(map(lambda x : "{}:{}".format(x[0], x[1]), replace.items()))
                            match[rxn_id_match].add((fn_hash_id, lib_fn_hash_id, hash_val, replace_str))
                            #print(rxn_id_match, fn_hash_id, lib_fn_hash_id, hash_val, t)
                
            
            #s_hash = self.func_hash(tstoichiometry)
            
            
            #for hash_type in s_hash:
            #    hash_val = s_hash[hash_type]
            #    for hash_type_lib in self.all_hashes:
            #        if hash_val in self.all_hashes[hash_type_lib]:
            #            for rxn_id_match in self.all_hashes[hash_type_lib][hash_val]:
            #                if not rxn_id_match in match:
            #                    match[rxn_id_match] = set()
            #                replace_str = ';'.join(map(lambda x : "{}:{}".format(x[0], x[1]), replace.items()))
            #                match[rxn_id_match].add((hash_type, hash_type_lib, hash_val, replace_str))
                
        return match
    
fn_rev_stoich = lambda cs : dict(map(lambda x : (x[0], -1 * x[1]), cs.items()))

hlib = CStoichiometryHashLibrary()
hlib.hash_functions = {
    'std' : lambda x : hash(frozenset(x.items())),
    'rev' : lambda x : hash(frozenset(fn_rev_stoich(x).items())),
}
rxn = modelseed_local.get_seed_reaction('rxn05469')
hlib.hash_stoichiometry(rxn.id, rxn.cstoichiometry)
hlib.match({('cpd00020', 'c'): -1.0,
 ('cpd00067', 'c'): -1.0,
 ('cpd00020', 'm'): 1.0,
 ('cpd00067', 'm'): 1.0})

{'rxn05469': {('rev', 'rev', -1216179813177492475, 'c:1;m:0'),
  ('rev', 'std', 4874071275187448717, 'c:0;m:1'),
  ('std', 'rev', -1216179813177492475, 'c:0;m:1'),
  ('std', 'std', 4874071275187448717, 'c:1;m:0')}}

In [29]:
fn_rev_stoich = lambda cs : dict(map(lambda x : (x[0], -1 * x[1]), cs.items()))

hlib = CStoichiometryHashLibrary()
hlib.hash_functions = {
    'std' : lambda x : hash(frozenset(x.items())),
    'rev' : lambda x : hash(frozenset(fn_rev_stoich(x).items())),
}
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    if not rxn.is_obsolete:
        hlib.hash_stoichiometry(rxn.id, rxn.cstoichiometry)

In [23]:
ooo = {}
for k in cpd_mapping:
    if len(cpd_mapping[k]) > 0:
        ooo[k] = list(cpd_mapping[k])[0]

In [24]:
from biosapi.bios_model_reaction import BiosModelReaction
exclude = set(['R_ATPM'])
for o in mapper.model_reactions:
    model_reaction = BiosModelReaction(o)
    if model_reaction.id.startswith('R_EX_'):
        exclude.add(model_reaction.id)

In [25]:
o = list(filter(lambda x : x['id'] == 'R_HKYNH', mapper.model_reactions))[0]
model_reaction = BiosModelReaction(o)
stoichiometry = mapper.model_reaction.get_stoichiometry()
cs = {}
for k in stoichiometry:
    cmp = sid_to_spi[k]['compartment']
    spi_id = sid_to_spi[k]['id']
    if spi_id in ooo:
        spi_id = ooo[spi_id]
    cs[(spi_id, cmp)] = stoichiometry[k]
cs

IndexError: list index out of range

In [396]:
for o in model_species:
    if not o['id'] in ooo:
        ooo[o['id']] = bigg_to_seed[o['id'][:-2][2:]]

In [336]:
ooo['M_thymd_c'] = 'cpd00184'

In [ ]:
#R_CHTNDA

In [30]:
predicted_rxn_mapping = {}
for o in mapper.model_reactions:
    model_reaction = BiosModelReaction(o)
    if not model_reaction.id in exclude:
        stoichiometry = model_reaction.get_stoichiometry()
        cs = {}
        for k in stoichiometry:
            cmp = sid_to_spi[k]['compartment']
            spi_id = sid_to_spi[k]['id']
            if spi_id in ooo:
                spi_id = ooo[spi_id]
            cs[(spi_id, cmp)] = stoichiometry[k]
        m = hlib.match(cs)
        if len(m) == 1:
            
            predicted_rxn_mapping[model_reaction.id] = list(m.keys())[0]
        elif len(m) > 1:
            print('*', model_reaction.id, list(m.keys()))
        else:
            print('X', model_reaction.id)

X RXN2T__45__476__45__L2R
X RXN2T__45__379__45__R2L
X RXN2T__45__434__45__R2L
X RXN0__45__1441__45__L2R
X VALINE__45____45__TRNA__45__LIGASE__45__RXN__45__L2R
X RXN2T__45__126__45__R2L
X PANTEPADENYLYLTRAN__45__RXN__45__L2R
X ADENODEAMIN__45__RXN__45__L2R
X RXN2T__45__234__45__L2R
X PRO__45__TRANS__45__RXN__45__R2L
X TRANS__45__RXN2T__45__316__45__R2L__91__CCO__45__GLYOXYSOME__93__
X P__45__AMINO__45__BENZOATE__45__TRANS__45__RXN__45__R2L
X TRYPTOPHAN__45__23__45__DIOXYGENASE__45__RXN__45__L2R
X PRODISULFREDUCT__45__A__45__RXN__45__L2R
X RXN3O__45__54__45__L2R
X RXN3DJ__45__11417__45__L2R
X L__45__LYSINE__45__AMINOTRANSFERASE__45__RXN__45__L2R
X RXN2T__45__423__45__L2R
X RXN2T__45__391__45__L2R
X RXN2T__45__429__45__L2R
X GLUTATHIONE__45__REDUCT__45__NADPH__45__RXN__45__R2L
X TRANS__45__RXN2T__45__302__45__L2R__91__CCO__45__GLYOXYSOME__93__
X TRANS__45__RXN2T__45__303__45__R2L__91__CCO__45__MIT__93__
X RXN2T__45__233__45__L2R
X RXN2T__45__291__45__L2R
X RXN1F__45__144__45__L2R
X RXN2T_

In [32]:
target_model_id = 'iJDZ836'
rxn_mapping = mapper.get_rxn_mapping('ModelSeedReaction')
for rxn_id in rxn_mapping:
    if rxn_id in predicted_rxn_mapping:
        seed_id = predicted_rxn_mapping[rxn_id]
        seed_rxn = modelseed_local.get_seed_reaction(seed_id)
        #print(seed_rxn.data['definition'])
        if len(rxn_mapping[rxn_id]) == 0:
            resp = bios.set_annotation_model_reaction(target_model_id, mapper.sid_to_rxn_id[rxn_id], 'ModelSeedReaction', predicted_rxn_mapping[rxn_id], 'auto', 3)
            pass
        elif len(rxn_mapping[rxn_id]) == 1:
            matche_prev = list(rxn_mapping[rxn_id])[0] == predicted_rxn_mapping[rxn_id]
            if matche_prev:
                #print('PREV:', matche_prev, rxn_id, rxn_mapping[rxn_id], predicted_rxn_mapping[rxn_id])
                pass
            else:
                resp = bios.set_annotation_model_reaction(target_model_id, mapper.sid_to_rxn_id[rxn_id], 'ModelSeedReaction', predicted_rxn_mapping[rxn_id], 'auto', 3)
                print('PREV:', matche_prev, rxn_id, rxn_mapping[rxn_id], predicted_rxn_mapping[rxn_id])
        else:
            print('OPT :', rxn_id, rxn_mapping[rxn_id], predicted_rxn_mapping[rxn_id])

OPT : CITSYN__45__RXN__45__L2R__91__CCO__45__MIT__93__ {'rxn30641', 'rxn00256'} rxn00256
OPT : ISOCITRATE__45__DEHYDROGENASE__45__NAD__43____45__RXN__45__L2R__91__CCO__45__MIT__93__ {'rxn30716', 'rxn15056'} rxn00505
OPT : ACONITATEHYDR__45__RXN__45__L2R__91__CCO__45__MIT__93__ {'rxn01388', 'rxn15280'} rxn01388
OPT : _3__46__6__46__3__46__6__45__RXN__45__L2R__91__CCO__45__EXTRACELLULAR__93__ {'rxn00062', 'rxn09694'} rxn09694
OPT : FUMHYDR__45__RXN__45__R2L__91__CCO__45__MIT__93__ {'rxn00799', 'rxn30674'} rxn00799
OPT : THREODEHYD__45__RXN__45__L2R {'rxn01068', 'rxn11130'} rxn01068
PREV: False RXN2T__45__64__45__L2R {'rxn30427'} rxn11720
OPT : PYRUVDEH__45__RXN__45__L2R__91__CCO__45__MIT__93__ {'rxn00154', 'rxn30753'} rxn00154
OPT : SUCCCOASYN__45__RXN__45__R2L__91__CCO__45__MIT__93__ {'rxn00285', 'rxn31087'} rxn00285
OPT : _2OXOGLUTARATEDEH__45__RXN__45__L2R__91__CCO__45__MIT__93__ {'rxn30621', 'rxn08094'} rxn08094
OPT : NGAM {'rxn00062', 'rxn09694'} rxn00062
OPT : F16BDEPHOS__45__RXN__

In [243]:
resp = set_annotation_model_reaction(bios, 'iMM904', 3154009, 'ModelSeedReaction', 'rxn00256', 'fliu', 5)
resp.content

http://192.168.1.15:8080/biosynth-web-biobase/api/model/annotation/iMM904/mrxn/3154009/ModelSeedReaction/rxn00256


b'9664790'

In [230]:
resp = set_annotation_model_species(bios, 'iMM904', 3153113, 'BiGGMetabolite', 'oaa', 'fliu', 5)

http://192.168.1.15:8080/biosynth-web-biobase/api/model/annotation/iMM904/spi/3153113/BiGGMetabolite/oaa


In [373]:
manual_mapping = {
    'cpd00013' : ['M_nh4_c', 'M_nh4_e', 'M_nh4_m', 'M_nh4_n', 'M_nh4_x'],
}

for k in ['M_nh4_c', 'M_nh4_e', 'M_nh4_m', 'M_nh4_n', 'M_nh4_x']:
    #print(k, sid_to_id[k])
    resp = set_annotation_model_species(bios, 'iMM904', sid_to_id[k], 'BiGGMetabolite', 'nh4', 'fliu', 5)
    resp = set_annotation_model_species(bios, 'iMM904', sid_to_id[k], 'ModelSeed', 'cpd00013', 'fliu', 5)

In [289]:
d.keys()

Index(['model_id', 'cpd00067', 'cpd00001', 'cpd00002', 'cpd00008', 'cpd00009',
       'cpd00012', 'cpd00006', 'cpd00005', 'cpd00003', 'cpd00004', 'cpd00011',
       'cpd00018', 'cpd00010', 'cpd00020', 'cpd00061', 'cpd00029', 'cpd11493',
       'cpd00013', 'cpd00007', 'cpd00025', 'cpd00242', 'cpd00022', 'cpd00014',
       'cpd00023', 'cpd00053', 'cpd00031', 'cpd00015', 'cpd00982', 'cpd00024',
       'cpd00106', 'cpd00036'],
      dtype='object')

3152871

'H+'

In [233]:
for k in id_sid:
    if id_sid[k] == 'R_CSm':
        print(k)

3154009


In [247]:
Databases.BIGG1_REACTION.value

'bigg1.reaction'

```java
 @RequestMapping(value="/{modelId}/cmp/{cmpId}/{scmp}", method=RequestMethod.PUT)
  public ResponseEntity<Object> putCompartmentAnnotation(@PathVariable String modelId, 
                                                         @PathVariable Long cmpId,
                                                         @PathVariable String scmp,
                                                         @RequestBody Map<String, Object> data) {
    logger.debug("{} -> {} -> [{}]{} -> {}", modelId, cmpId, scmp);
    ExtendedMetabolicModelEntity model = metabolicModelService.getMetabolicModel(modelId);
    if (model == null) {
      return new ResponseEntity<>(HttpStatus.NOT_FOUND);
    }
    SubcellularCompartment compartment = SubcellularCompartment.valueOf(scmp);
//    if (cpd == null) {
//      return new ResponseEntity<>(HttpStatus.NOT_FOUND);
//    }
    if (data == null || !data.containsKey("user") || !data.containsKey("score")) {
      return new ResponseEntity<>(HttpStatus.BAD_REQUEST);
    }
    String user = (String) data.get("user").toString();
    int score = Integer.parseInt(data.get("score").toString());
    Object result = annotationService.addAnnotationToModelCompartment(
        model.getId(), cmpId, compartment, user, score);
    logger.debug("{}", result);
    return new ResponseEntity<>(result, HttpStatus.OK);
  }
```